In [ ]:
import pickle
from matplotlib import pyplot as plt
import numpy as np
import json
from sklearn.metrics.pairwise import euclidean_distances
import cv2
from PIL import Image

In [ ]:
matlab_points = pickle.load(open('/root/data/alok/biomass_estimation/playground/image_to_checkerboard_points_matlab_bolaks.pkl', 'rb'))


In [ ]:
left_y_coordinates, right_y_coordinates = [], []
for key in list(matlab_points.keys()):
    left_y_coordinates.extend(list(matlab_points[key]['left_corners'][:, 1]))
    right_y_coordinates.extend(list(matlab_points[key]['right_corners'][:, 1]))
left_y_coordinates = np.array(left_y_coordinates)
right_y_coordinates = np.array(right_y_coordinates)
print('Mean y-coordinate deviation: {}'.format(np.mean(left_y_coordinates - right_y_coordinates)))

In [ ]:
matlab_df = pickle.load(open('/root/data/alok/biomass_estimation/playground/analysis_df_bolaks.pkl', 'rb'))

In [ ]:
plt.figure(figsize=(20, 10))
plt.hist(matlab_df[matlab_df.deviation.abs() < 10].deviation)
plt.xlabel('Deviation (mm)')
plt.ylabel('Frequency')
plt.grid()
plt.show()

In [ ]:
left_y_coordinates - right_y_coordinates

In [ ]:
(matlab_left - matlab_right) - (circular_left - circular_right)

In [ ]:
matlab_wcm = pickle.load(open('/root/data/alok/biomass_estimation/playground/wcm_matlab.pkl', 'rb'))
circular_wcm = pickle.load(open('/root/data/alok/biomass_estimation/playground/wcm_circular.pkl', 'rb'))

In [ ]:
diffs = np.squeeze(1e3 * (euclidean_distances(world_coordinate_matrix_matlab) - euclidean_distances(world_coordinate_matrix_circular))).flatten()

In [ ]:
ds = []
for i in range(world_coordinate_matrix_matlab.shape[0]):
    for j in range(i+1, world_coordinate_matrix_matlab.shape[0]):
        d = np.linalg.norm(world_coordinate_matrix_matlab[i, :] - world_coordinate_matrix_matlab[j, :])
        ds.append(d)
        
ds = np.array(ds)
diffs = 1e3 * ds - matlab_df.ground_truth_distance_mms.values
plt.hist(diffs)
plt.show()

In [ ]:
circular_params

In [ ]:
ds = []
for i in range(world_coordinate_matrix_circular.shape[0]):
    for j in range(i+1, world_coordinate_matrix_circular.shape[0]):
        d = np.linalg.norm(world_coordinate_matrix_circular[i, :] - world_coordinate_matrix_circular[j, :])
        ds.append(d)
        
ds = np.array(ds)
diffs = 1e3 * ds - circular_df.ground_truth_distance_mms.values
plt.hist(diffs)
plt.show()

In [ ]:
diffs = np.squeeze(1e3 * (euclidean_distances(matlab_wcm[0]) - euclidean_distances(circular_wcm[0]))).flatten()

In [ ]:
plt.hist(diffs)
plt.show()

In [ ]:
matlab_params = json.load(open('/root/data/alok/biomass_estimation/playground/2020-01-14T00_00_00Z_L40013180_R40029775_stereo-parameters.json'))
circular_params = json.load(open('/root/data/alok/biomass_estimation/playground/EstimatedStereoCamParams_L40013180_R40029775.json'))

In [ ]:
matlab_params['TranslationOfCamera2']

In [ ]:
circular_params['TranslationOfCamera2']

In [ ]:
matlab_params['CameraParameters1']['FocalLength']

In [ ]:
circular_params['CameraParameters1']['FocalLength']

In [ ]:
(matlab_params['CameraParameters1']['FocalLength'][0] * matlab_params['TranslationOfCamera2'][0]) / 608

In [ ]:
circular_params['CameraParameters1']['FocalLength'][0] * circular_params['TranslationOfCamera2'][0] / 600

In [ ]:
plt.figure(figsize=(20, 10))
plt.hist(matlab_df[matlab_df.deviation.abs() < 10].deviation)
plt.xlabel('Deviation (mm)')
plt.ylabel('Frequency')
plt.grid()
plt.show()

In [ ]:
matlab_df = pickle.load(open('/root/data/alok/biomass_estimation/playground/analysis_df_bolaks.pkl', 'rb'))


In [ ]:
plt.hist(matlab_df.predicted_distance_mms - matlab_df.ground_truth_distance_mms, bins=100)
plt.show()

In [ ]:
plt.hist(circular_df.predicted_distance_mms - circular_df.ground_truth_distance_mms)
plt.show()

In [ ]:
((0.605)**3 - (0.60)**3) / (0.60)**3

In [ ]:
circular_cameraMatrix1 = np.array(circular_params['CameraParameters1']['IntrinsicMatrix']).transpose() 
circular_cameraMatrix2 = np.array(circular_params['CameraParameters2']['IntrinsicMatrix']).transpose()

In [ ]:
matlab_cameraMatrix1 = np.array(matlab_params['CameraParameters1']['IntrinsicMatrix']).transpose() 
matlab_cameraMatrix2 = np.array(matlab_params['CameraParameters2']['IntrinsicMatrix']).transpose()
R = np.array(matlab_params['RotationOfCamera2']).transpose()
T = np.array(matlab_params['TranslationOfCamera2']).transpose()
world_checkerboard_points_list = []
for left_corners, right_corners in zip(matlab_left, matlab_right):
    P1 = np.dot(matlab_cameraMatrix1, np.hstack([np.eye(3), np.array([[0], [0], [0]])]))
    P2 = np.dot(matlab_cameraMatrix2, np.hstack([R, T[:, np.newaxis]]))
    wcp = cv2.triangulatePoints(P1, P2, left_corners, right_corners).T
    world_checkerboard_points = wcp[:,:3] / wcp[:,3, None]
    world_checkerboard_points_list.append(world_checkerboard_points)
    
world_coordinate_matrix_matlab = np.empty([len(world_checkerboard_points_list), 3])
for i, world_checkerboard_points in enumerate(world_checkerboard_points_list):
        world_coordinate_matrix_matlab[i, :] = world_checkerboard_points

world_coordinate_matrix_matlab = np.array([world_coordinate_matrix_matlab[:, 0],
                                           world_coordinate_matrix_matlab[:, 2],
                                           -world_coordinate_matrix_matlab[:, 1]]).T * 1e-3

In [ ]:
circular_cameraMatrix1 = np.array(circular_params['CameraParameters1']['IntrinsicMatrix']).transpose() 
circular_cameraMatrix2 = np.array(circular_params['CameraParameters2']['IntrinsicMatrix']).transpose()
R = np.array(circular_params['RotationOfCamera2']).transpose()
T = np.array(circular_params['TranslationOfCamera2']).transpose()
world_checkerboard_points_list = []
for left_corners, right_corners in zip(circular_left, circular_right):
    P1 = np.dot(circular_cameraMatrix1, np.hstack([np.eye(3), np.array([[0], [0], [0]])]))
    P2 = np.dot(circular_cameraMatrix2, np.hstack([R, T[:, np.newaxis]]))
    wcp = cv2.triangulatePoints(P1, P2, left_corners, right_corners).T
    world_checkerboard_points = wcp[:,:3] / wcp[:,3, None]
    world_checkerboard_points_list.append(world_checkerboard_points)
    
world_coordinate_matrix_circular = np.empty([len(world_checkerboard_points_list), 3])
for i, world_checkerboard_points in enumerate(world_checkerboard_points_list):
        world_coordinate_matrix_circular[i, :] = world_checkerboard_points

world_coordinate_matrix_circular = np.array([world_coordinate_matrix_circular[:, 0],
                                             world_coordinate_matrix_circular[:, 2],
                                             -world_coordinate_matrix_circular[:, 1]]).T * 1e-3

In [ ]:
(world_coordinate_matrix_circular - circular_wcm) * 1e3

In [ ]:
#!/usr/bin/env python

'''
Check whether calibration is valid
'''
import sys
from glob import glob, iglob
import os
import os.path
import re
import math
import random
import shutil
import json
import signal
import numpy as np
import cv2
import pandas as pd
from matplotlib import pyplot as plt
import click
import logging
import datetime
import warnings
import pickle

log = logging.getLogger("calidate")
np.seterr(all = 'raise')

def signal_handler(signal, frame):
    print('Ctrl-C .... Exiting ...')
    sys.exit(0)

def create_output_dir(dir_to_create):
    try:
        os.makedirs(dir_to_create)
    except FileExistsError:
        print(f'Directory already exists {dir_to_create} skip creating it')
    except Exception as e:
        log.error(f"Error creating {dir_to_create} Error: {e}")
        raise

def rm_output_dir(dir_to_remove):
    try:
        shutil.rmtree(dir_to_remove)
    except FileNotFoundError:
        print(f'Directory already removed {dir_to_remove} skipping')
    except Exception as e:
        log.error(f"Error removing {dir_to_remove} Error: {e}")
        raise

def load_params(params_file):
    try:
        with open(params_file, 'r') as f:
            params = json.load(f)
            return params
    except FileNotFoundError:
        log.error(f"File {params_file} not found. Exiting...")
        sys.exit(-1)

def rectify(params):
    print('Rectifying')
    cameraMatrix1 = np.array(params['CameraParameters1']['IntrinsicMatrix']).transpose()
    cameraMatrix2 = np.array(params['CameraParameters2']['IntrinsicMatrix']).transpose()
    distCoeffs1 = params['CameraParameters1']['RadialDistortion'][0:2] + \
    params['CameraParameters1']['TangentialDistortion'] + \
        [params['CameraParameters1']['RadialDistortion'][2]]
    distCoeffs1 = np.array(distCoeffs1)
    distCoeffs2 = params['CameraParameters2']['RadialDistortion'][0:2] + \
    params['CameraParameters2']['TangentialDistortion'] + \
        [params['CameraParameters2']['RadialDistortion'][2]]
    distCoeffs2 = np.array(distCoeffs2)
    R = np.array(params['RotationOfCamera2']).transpose()
    T = np.array(params['TranslationOfCamera2']).transpose()
    imageSize = (4096, 3000)

#    left_rectified = cv2.undistort(left_im, cameraMatrix1, distCoeffs1, None, newcameramtx1)
#    right_rectified = cv2.undistort(right_im, cameraMatrix2, distCoeffs2, None, newcameramtx2)
    # perform rectification
    (R1, R2, P1, P2, Q, leftROI, rightROI) = cv2.stereoRectify(cameraMatrix1, distCoeffs1, cameraMatrix2, distCoeffs2, imageSize, R, T, None, None, None, None, None, cv2.CALIB_ZERO_DISPARITY, 0)
    left_maps = cv2.initUndistortRectifyMap(cameraMatrix1, distCoeffs1, R1, P1, imageSize, cv2.CV_16SC2)
    right_maps = cv2.initUndistortRectifyMap(cameraMatrix2, distCoeffs2, R2, P2, imageSize, cv2.CV_16SC2)
    return left_maps, right_maps

def remap_pair(left_img_path, right_img_path, left_maps, right_maps):
    img_left = cv2.imread(left_img_path)
    img_right = cv2.imread(right_img_path)
    remap_left = cv2.remap(img_left, left_maps[0], left_maps[1], cv2.INTER_LANCZOS4)
    remap_right = cv2.remap(img_right, right_maps[0], right_maps[1], cv2.INTER_LANCZOS4)
    output_left_dir = os.path.dirname(left_img_path).replace('good_calibration_images', 'good_calibration_images_rectified')
    output_right_dir = os.path.dirname(right_img_path).replace('good_calibration_images', 'good_calibration_images_rectified')
    if not os.path.exists(output_left_dir):
        os.makedirs(output_left_dir)
    if not os.path.exists(output_right_dir):
        os.makedirs(output_right_dir)
    Image.fromarray(remap_left).save(left_img_path.replace('good_calibration_images', 'good_calibration_images_rectified'))
    Image.fromarray(remap_right).save(right_img_path.replace('good_calibration_images', 'good_calibration_images_rectified'))
    return remap_left, remap_right

def find_corners(remap):
    gray = cv2.cvtColor(remap,cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (1024, 750))
    ret, resized_corners = cv2.findChessboardCorners(resized, (9,6), None)
    if ret:
        corners = resized_corners * 4
        adj_corners = cv2.cornerSubPix(gray, corners, (5, 5), (-1, -1), (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001))
        return np.squeeze(adj_corners)
    else:
        return None

def checkerboard_points(validation_dir, maps, img_format):
    left_maps, right_maps = maps
    image_to_checkerboard_points = {}
    left_image_fs = glob(os.path.join(validation_dir, 'left', '*left_*.' + img_format))
    for i, left_image_f in enumerate(left_image_fs):
        right_image_f = left_image_f.replace('left', 'right')
        if not os.path.exists(right_image_f):
            log.error(f'Path does not exist: {right_image_f}')
            continue

        remap_left, remap_right = remap_pair(left_image_f, right_image_f, left_maps, right_maps)
        print(f'{i} Finding corners for {left_image_f} pair')
        left_corners = find_corners(remap_left)
        right_corners = find_corners(remap_right)
        if left_corners is None:
            print(f'{i} Failed finding corners for {left_image_f}')
            cv2.imwrite(validation_dir + 'left_' + str(i) + '.png', remap_left)
        if right_corners is None:
            print(f'{i} Failed finding corners for {right_image_f}')
            cv2.imwrite(validation_dir + 'right_' + str(i) + '.png', remap_right)
        if left_corners is not None and right_corners is not None:
            print(f'{i} left corners {len(left_corners)} right corners {len(right_corners)}')
            image_to_checkerboard_points[left_image_f] = {}
            image_to_checkerboard_points[left_image_f]['left_corners'] = left_corners
            image_to_checkerboard_points[left_image_f]['right_corners'] = right_corners
            cwd = os.getcwd()
            rectify_dir = os.path.join(cwd, 'rectify')
            cv2.imwrite(rectify_dir + '/' + str(i) + '_left.png', remap_left)
            cv2.imwrite(rectify_dir + '/' + str(i) + '_right.png', remap_right)


    return image_to_checkerboard_points

def convert_to_world_point(focal_len_pxl, x, y, d):
    PIXEL_SIZE_M = 3.45 * 1e-6
    FOCAL_LENGTH = focal_len_pxl * PIXEL_SIZE_M
    IMAGE_SENSOR_WIDTH = 0.01412
    IMAGE_SENSOR_HEIGHT = 0.01035
    PIXEL_COUNT_WIDTH = 4096
    PIXEL_COUNT_HEIGHT = 3000
    image_center_x = PIXEL_COUNT_WIDTH / 2.0
    image_center_y = PIXEL_COUNT_HEIGHT / 2.0
    px_x = x - image_center_x
    px_z = image_center_y - y

    sensor_x = px_x * (IMAGE_SENSOR_WIDTH / 4096)
    sensor_z = px_z * (IMAGE_SENSOR_HEIGHT / 3000)

    world_y = d
    world_x = (world_y * sensor_x) / FOCAL_LENGTH
    world_z = (world_y * sensor_z) / FOCAL_LENGTH
    return np.array([world_x, world_y, world_z])

def depth_from_disp(stereo_params, disp):
    FOCAL_LENGTH_PIXEL = stereo_params['CameraParameters1']['FocalLength'][0]
    BASELINE = abs(stereo_params['TranslationOfCamera2'][0] / 1e3) # convert millimeters to meters and use absolute value
    FOCAL_LENGTH_PIXEL = stereo_params['CameraParameters1']['FocalLength'][0]
    depth = FOCAL_LENGTH_PIXEL*BASELINE / np.array(disp)
    return depth

def euclidean_distance(p1, p2):
    try:
        dist = ((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2 + (p1[2] - p2[2])**2)**0.5
    except FloatingPointError as e:
        #print(f'Error calculating eucl distance between Pt1 {p1}  Pt2 {p2}')
        dist = 0.0
    return dist

def distance_between_positions(i, j):
    CHECKERBOARD_SIDE_LENGTH = 0.049294
    p1_ax_0_pos = i // 9
    p1_ax_1_pos = i % 9
    p2_ax_0_pos = j // 9
    p2_ax_1_pos = j % 9
    return CHECKERBOARD_SIDE_LENGTH * ((p1_ax_0_pos - p2_ax_0_pos)**2 + (p1_ax_1_pos - p2_ax_1_pos)**2)**0.5

def world_coordinate(stereo_params, image_to_checkerboard_points):
    print(f'Converting to world coords')
    focal_len_pxl = stereo_params['CameraParameters1']['FocalLength'][0]
    world_coordinate_matrix = np.empty([len(list(image_to_checkerboard_points.keys())), 54, 3])#, dtype=np.longdouble)
    for idx, left_image_f in enumerate(image_to_checkerboard_points.keys()):
        print(idx)
        left_corners = image_to_checkerboard_points[left_image_f]['left_corners']
        right_corners = image_to_checkerboard_points[left_image_f]['right_corners']
    for i in range(54):
        disp = abs(left_corners[i][0] - right_corners[i][0])
        depth = depth_from_disp(stereo_params, disp)
        world_coordinate_matrix[idx, i, :] = convert_to_world_point(focal_len_pxl, left_corners[i][0], left_corners[i][1], depth)

    return world_coordinate_matrix

def analysis(world_coordinate_matrix):
    analysis_df = pd.DataFrame()
    print(f'Analysis... Shape: {world_coordinate_matrix.shape}')
    for idx in range(world_coordinate_matrix.shape[0]):
        for i in range(world_coordinate_matrix.shape[1]-1):
            for j in range(i+1, world_coordinate_matrix.shape[1]):
                p1 = world_coordinate_matrix[idx, i, :]
                p2 = world_coordinate_matrix[idx, j, :]
                predicted_distance = euclidean_distance(p1, p2)
                ground_truth_distance = distance_between_positions(i, j)
                row = {
                    'checkerboard_idx': idx,
                    'predicted_distance_mms': predicted_distance * 1000,
                    'ground_truth_distance_mms': ground_truth_distance * 1000
                }
                analysis_df = analysis_df.append(row, ignore_index=True)
                analysis_df['deviation'] = analysis_df['predicted_distance_mms'] - analysis_df['ground_truth_distance_mms']
                analysis_df['pct_deviation'] = analysis_df['deviation'] / analysis_df['ground_truth_distance_mms']
    pickle.dump(analysis_df, open('/root/data/alok/biomass_estimation/playground/analysis_df_bolaks.pkl', 'wb'))
    return analysis_df, world_coordinate_matrix


def calidate(folder, stereo_params, save_to, img_format):

    print(f'Validating Calibration using Images in directory: {folder} Params: {stereo_params}')
    # cleanup first
    cwd = os.getcwd()
    rm_output_dir(cwd + '/rectify')
    create_output_dir(cwd + '/rectify')
    params_json = load_params(stereo_params)
    left_maps, right_maps = rectify(params_json)
    image_check_pts = checkerboard_points(folder, (left_maps, right_maps), img_format)
    if image_check_pts == {}:
        log.error(f'Failed finding checkerboard points. Exiting...')
        sys.exit(-1)
    world_coords = world_coordinate(params_json, image_check_pts)
    analysis_df, world_coordinate_matrix = analysis(world_coords)
    print('Finished calibration validation')

    return analysis_df, image_check_pts, world_coordinate_matrix


In [ ]:
folder = '/root/data/alok/biomass_estimation/playground/bolaks/good_calibration_images/test'
stereo_params = '/root/data/s3/aquabyte-stereo-parameters/L40039154_R40012648/2020-02-04T00:00:00Z_L40039154_R40012648_stereo-parameters.json'
save_to = '/root/data/alok/biomass_estimation/playground/results_bolaks.png'
img_format = 'png'
analysis_df, image_check_pts, world_coordinate_matrix = calidate(folder, stereo_params, save_to, img_format)

In [ ]:
plt.figure(figsize=(20, 10))
plt.hist(analysis_df.deviation, bins=20)
plt.grid()
plt.show()


In [ ]:
folder = '/root/data/alok/biomass_estimation/playground/bolaks/good_calibration_images/test'
stereo_params = '/root/data/s3/aquabyte-stereo-parameters/L40039154_R40012648/2020-02-04T00:00:00Z_L40039154_R40012648_stereo-parameters.json'
save_to = '/root/data/alok/biomass_estimation/playground/results_bolaks.png'
img_format = 'png'
analysis_df, image_check_pts, world_coordinate_matrix = calidate(folder, stereo_params, save_to, img_format)

In [ ]:
plt.figure(figsize=(20, 10))
plt.hist(analysis_df.deviation, bins=20)
plt.grid()
plt.show()


In [ ]:
analysis_df

In [ ]:
analysis_df_bassem = pickle.load(open('/root/data/alok/biomass_estimation/playground/analysis_df_bassem.pkl', 'rb'))

In [ ]:
np.min(analysis_df_bassem[0][0][analysis_df_bassem[0][0].predicted_distance_mms > 0].deviation)

In [ ]:
analysis_df.deviation.min()

In [ ]:
left_y_coordinates, right_y_coordinates = [], []
for key in list(image_check_pts.keys()):
    left_y_coordinates.extend(list(matlab_points[key]['left_corners'][:, 1]))
    right_y_coordinates.extend(list(matlab_points[key]['right_corners'][:, 1]))
left_y_coordinates = np.array(left_y_coordinates)
right_y_coordinates = np.array(right_y_coordinates)
print('Mean y-coordinate deviation: {}'.format(np.mean(left_y_coordinates - right_y_coordinates)))

In [ ]:
plt.figure(figsize=(20, 10))
plt.hist(left_y_coordinates - right_y_coordinates, bins=50)
plt.show()